## Deliverable 2. Create a Customer Travel Destinations Map.

In [13]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [18]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kapaa,US,22.0752,-159.3190,79.12,67,0,12.66,clear sky
1,1,Dikson,RU,73.5069,80.5464,-0.94,98,100,7.14,overcast clouds
2,2,Saldanha,ZA,-33.0117,17.9442,62.15,70,0,27.22,clear sky
3,3,Andenes,NO,69.3143,16.1194,40.98,75,75,11.50,light rain
4,4,Breznice,CZ,49.5576,13.9506,40.33,94,0,2.46,clear sky


In [19]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [20]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kapaa,US,22.0752,-159.3190,79.12,67,0,12.66,clear sky
5,5,Hilo,US,19.7297,-155.0900,77.61,77,55,4.00,broken clouds
8,8,Georgetown,MY,5.4112,100.3354,78.73,89,20,0.00,few clouds
9,9,Vaini,TO,-21.2000,-175.2000,82.56,89,75,2.30,broken clouds
10,10,Shwebo,MM,22.5667,95.7000,83.50,38,6,6.67,clear sky
11,11,Faanui,PF,-16.4833,-151.7500,81.48,76,27,12.77,light rain
16,16,Tucuman,AR,-26.8241,-65.2226,80.71,50,0,4.90,clear sky
19,19,Pemangkat,ID,1.1667,108.9667,76.96,87,93,3.53,overcast clouds
20,20,Pisco,PE,-13.7000,-76.2167,79.27,60,99,9.93,overcast clouds
27,27,Rikitea,PF,-23.1203,-134.9692,79.48,75,100,14.88,light rain


In [21]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                206
City                   206
Country                206
Lat                    206
Lng                    206
Max Temp               206
Humidity               206
Cloudiness             206
Wind Speed             206
Current Description    206
dtype: int64

In [24]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kapaa,US,22.0752,-159.3190,79.12,67,0,12.66,clear sky
5,5,Hilo,US,19.7297,-155.0900,77.61,77,55,4.00,broken clouds
8,8,Georgetown,MY,5.4112,100.3354,78.73,89,20,0.00,few clouds
9,9,Vaini,TO,-21.2000,-175.2000,82.56,89,75,2.30,broken clouds
10,10,Shwebo,MM,22.5667,95.7000,83.50,38,6,6.67,clear sky


In [25]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kapaa,US,79.12,clear sky,22.0752,-159.3190,
5,Hilo,US,77.61,broken clouds,19.7297,-155.0900,
8,Georgetown,MY,78.73,few clouds,5.4112,100.3354,
9,Vaini,TO,82.56,broken clouds,-21.2000,-175.2000,
10,Shwebo,MM,83.50,clear sky,22.5667,95.7000,
11,Faanui,PF,81.48,light rain,-16.4833,-151.7500,
16,Tucuman,AR,80.71,clear sky,-26.8241,-65.2226,
19,Pemangkat,ID,76.96,overcast clouds,1.1667,108.9667,
20,Pisco,PE,79.27,overcast clouds,-13.7000,-76.2167,
27,Rikitea,PF,79.48,light rain,-23.1203,-134.9692,


In [33]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    print(hotels)
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_att

{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_att

{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}
Hotel not found... skipping.
{'html_att

In [32]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kapaa,US,79.12,clear sky,22.0752,-159.3190,
5,Hilo,US,77.61,broken clouds,19.7297,-155.0900,
8,Georgetown,MY,78.73,few clouds,5.4112,100.3354,
9,Vaini,TO,82.56,broken clouds,-21.2000,-175.2000,
10,Shwebo,MM,83.50,clear sky,22.5667,95.7000,
...,...,...,...,...,...,...,...
646,Palmas,BR,85.98,thunderstorm,-10.2128,-48.3603,
651,San Lorenzo,PY,81.59,clear sky,-25.3333,-57.5333,
652,Kalmunai,LK,76.89,overcast clouds,7.4167,81.8167,
653,Kassala,SD,77.58,overcast clouds,15.4510,36.4000,


In [10]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
